In [6]:
import data_utils_adv
import numpy as np
from sklearn.model_selection import KFold
from qwk import quadratic_weighted_kappa
import tensorflow as tf
import time
import os
import sys
import pandas as pd

print( 'start to load flags\n')

# flags
# tf.flags.DEFINE_float("epsilon", 0.1, "Epsilon value for Adam Optimizer.")
# tf.flags.DEFINE_float("l2_lambda", 0.3, "Lambda for l2 loss.")
# tf.flags.DEFINE_float("learning_rate", 0.001, "Learning rate")
# tf.flags.DEFINE_float("max_grad_norm", 10.0, "Clip gradients to this norm.")
# tf.flags.DEFINE_float("keep_prob", 0.9, "Keep probability for dropout")
# tf.flags.DEFINE_integer("evaluation_interval", 5, "Evaluate and print results every x epochs")
# tf.flags.DEFINE_integer("batch_size", 15, "Batch size for training.")
# tf.flags.DEFINE_integer("feature_size", 100, "Feature size")
# tf.flags.DEFINE_integer("num_samples", 1, "Number of samples selected from training for each score")
# tf.flags.DEFINE_integer("hops", 3, "Number of hops in the Memory Network.")
# tf.flags.DEFINE_integer("epochs", 200, "Number of epochs to train for.")
# tf.flags.DEFINE_integer("embedding_size", 300, "Embedding size for embedding matrices.")
# tf.flags.DEFINE_integer("essay_set_id", 1, "essay set id, 1 <= id <= 8")
# tf.flags.DEFINE_integer("token_num", 42, "The number of token in glove (6, 42)")
# tf.flags.DEFINE_boolean("gated_addressing", False, "Simple gated addressing")
# tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
# tf.flags.DEFINE_boolean("is_regression", False, "The output is regression or classification")
# tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")
# # hyper-parameters
# FLAGS = tf.flags.FLAGS

early_stop_count = 0
max_step_count = 10
is_regression = False
gated_addressing = False
essay_set_id = 1
batch_size = 15
embedding_size = 300
feature_size = 100
l2_lambda = 0.3
hops = 3
reader = 'bow' # gru may not work
epochs = 200
num_samples = 1
num_tokens = 42
test_batch_size = batch_size
random_state = 0

start to load flags



In [7]:
if is_regression:
    from memn2n_kv_regression import MemN2N_KV
else:
    from memn2n_kv import MemN2N_KV
# print flags info
orig_stdout = sys.stdout
timestamp = time.strftime("%b_%d_%Y_%H:%M:%S", time.localtime())
folder_name = 'essay_set_{}_cv_{}_{}'.format(essay_set_id, num_samples, timestamp)
out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs/adversary_training/", folder_name))
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# save output to a file
#f = file(out_dir+'/out.txt', 'w')
#sys.stdout = f
print("Writing to {}\n".format(out_dir))

# print("\nParameters:")
# for key in sorted(FLAGS.__flags.keys()):
#     print("{}={}".format(key, getattr(FLAGS, key)))
# print("")

# with open(out_dir+'/params', 'w') as f:
#     for key in sorted(FLAGS.__flags.keys()):
#         f.write("{}={}".format(key, getattr(FLAGS, key)))
#         f.write("\n")

# hyper-parameters end here
training_path = 'train_adv/prompt1/noDisfluency&grammar_1_train_valid_reduce.csv'
essay_list, resolved_scores, essay_id = data_utils_adv.load_training_data(training_path, essay_set_id)

# print(essay_id)
testing_path = 'aes_data/essay1/fold_0/test.txt'
essay_list_test, resolved_scores_test, essay_id_test = data_utils_adv.load_testing_data(testing_path, essay_set_id)

max_score = max(resolved_scores)
min_score = min(resolved_scores)
if essay_set_id == 7:
    min_score, max_score = 0, 30
elif essay_set_id == 8:
    min_score, max_score = 0, 60

print( 'max_score is {} \t min_score is {}\n'.format(max_score, min_score))
with open(out_dir+'/params', 'a') as f:
    f.write('max_score is {} \t min_score is {} \n'.format(max_score, min_score))


Writing to /mnt/data/rajivratn/memory_networks/automated-essay-grading/runs/adversary_training/essay_set_1_cv_1_Mar_25_2020_21:21:22

                                               names  essay_id
0  8, "Dear local newspaper, I think effects comp...         1
1  10, "Dear Local Newspaper, @CAPS1 I have found...         2
2  9, "Dear reader, @ORGANIZATION1 has had a dram...         3
3  10, "I agree that computers deffinately are an...         4
4  8, "Dear local Newspaper, @CAPS1 in the societ...         5
max_score is 12 	 min_score is 2



In [8]:
score_range = range(min_score, max_score+1)

#word_idx, _ = data_utils.build_vocab(essay_list, vocab_limit)

# load glove
word_idx, word2vec = data_utils_adv.load_glove(num_tokens, dim=embedding_size)

vocab_size = len(word_idx) + 1
# stat info on data set

sent_size_list = list(map(len, [essay for essay in essay_list]))
# print("sent size list", sent_size_list)
max_sent_size = max(sent_size_list)
mean_sent_size = int(np.mean(sent_size_list))

print( 'max sentence size: {} \nmean sentence size: {}\n'.format(max_sent_size, mean_sent_size))
with open(out_dir+'/params', 'a') as f:
    f.write('max sentence size: {} \nmean sentence size: {}\n'.format(max_sent_size, mean_sent_size))

print( 'The length of score range is {}'.format(len(score_range)))
E = data_utils_adv.vectorize_data(essay_list, word_idx, max_sent_size)

labeled_data = zip(E, resolved_scores, sent_size_list)


==> glove is loaded
max sentence size: 911 
mean sentence size: 394

The length of score range is 11


In [9]:
# For Testing
sent_size_list_T = list(map(len, [essayT for essayT in essay_list_test]))
max_sent_size_T = max(sent_size_list_T)
mean_sent_size_T = int(np.mean(sent_size_list_T))

# max_sent_size_T = max_sent_size_T.ljust(122, ' ')
print( 'max sentence size: {} \nmean sentence size: {}\n'.format(max_sent_size_T, mean_sent_size_T))
with open(out_dir+'/params', 'a') as f:
    f.write('max sentence size: {} \nmean sentence size: {}\n'.format(max_sent_size_T, mean_sent_size_T))

# print( 'The length of score range of Testing Daat is {}'.format(len(score_range)))
E_T = data_utils_adv.vectorize_data(essay_list_test, word_idx, max_sent_size_T)

labeled_data_T = zip(E_T, resolved_scores_test, sent_size_list_T)


max sentence size: 811 
mean sentence size: 384



In [10]:
def train_step(m, e, s, ma):
    start_time = time.time()
    feed_dict = {
        model._query: e,
        model._memory_key: m,
        model._score_encoding: s,
        model._mem_attention_encoding: ma,
        model.keep_prob: 0.9
        #model.w_placeholder: word2vec
    }
    _, step, predict_op, cost = sess.run([train_op, global_step, model.predict_op, model.cost], feed_dict)
    end_time = time.time()
    time_spent = end_time - start_time
    return predict_op, cost, time_spent

def test_step(e, m):
    feed_dict = {
        model._query: e,
        model._memory_key: m,
        model.keep_prob: 1
        #model.w_placeholder: word2vec
    }
    preds, mem_attention_probs = sess.run([model.predict_op, model.mem_attention_probs], feed_dict)
    if is_regression:
        preds = np.clip(np.round(preds), min_score, max_score)
        return preds, mem_attention_probs
    else:
        return preds, mem_attention_probs


In [12]:
fold_count = 0
# kf = KFold(n_splits=2, random_state=random_state)
early_stop_count = 0

trainE = []
train_scores = []
train_essay_id = []
best_kappa_scores = []

testE = []
test_scores = []
test_essay_id = []

print(len(E_T))
# print(len(resolved_scores_test))
# print(len(essay_list_test))
for test_index in range(len(essay_id_test)):
    testE.append(E[test_index])
#     print(len(testE))
    test_scores.append(resolved_scores_test[test_index])
    test_essay_id.append(essay_id_test[test_index])

for train_index in range(len(essay_id)):
#     print("Train_index", train_index)
#     result = [int(x) for x, in train_index[0]]
#     print(result)
#     early_stop_count = 0
#     fold_count += 1
#     trainE = []
#     testE = []
#     train_scores = []
#     test_scores = []
#     train_essay_id = []
#     test_essay_id = []

    trainE.append(E[train_index])
    train_scores.append(resolved_scores[train_index])
    train_essay_id.append(essay_id[train_index])
    
#     for ite in train_index[0]:
# #         print(ite)
#         trainE.append(E[ite])
#         train_scores.append(resolved_scores[ite])
#         train_essay_id.append(essay_id[ite])
#         print("Training Samples", E[ite], resolved_scores[ite], essay_id[ite])
        
#     for ite in test_index:
#         testE.append(E[ite])
#         test_scores.append(resolved_scores[ite])
#         test_essay_id.append(essay_id[ite])
    
#trainE, testE, train_scores, test_scores, train_essay_id, test_essay_id = cross_validation.train_test_split(
#    E, resolved_scores, essay_id, test_size=.2, random_state=random_state)

memory = []
memory_score = []
memory_sent_size = []
memory_essay_ids = []
# pick sampled essay for each score
for i in score_range:
# test point: limit the number of samples in memory for 8
    for j in range(num_samples):
        if i in train_scores:
            score_idx = train_scores.index(i)
            score = train_scores.pop(score_idx)
            essay = trainE.pop(score_idx)
            sent_size = sent_size_list.pop(score_idx)
            memory.append(essay)
            memory_score.append(score)
            memory_essay_ids.append(train_essay_id.pop(score_idx))
            memory_sent_size.append(sent_size)
memory_size = len(memory)
if is_regression:
# bad naming
    train_scores_encoding = train_scores
else:
    train_scores_encoding = list(map(lambda x: score_range.index(x), train_scores))

    # data size
n_train = len(trainE)
n_test = len(testE)

print( 'The size of training data: {}'.format(n_train))
print( 'The size of testing data: {}'.format(n_test))
with open(out_dir+'/params_'.format(names), 'a') as f:
    f.write('The size of training data: {}\n'.format(n_train))
    f.write('The size of testing data: {}\n'.format(n_test))
    f.write('\nEssay scores in memory:\n{}'.format(memory_score))
    f.write('\nEssay ids in memory:\n{}'.format(memory_essay_ids))
    f.write('\nEssay ids in training:\n{}'.format(train_essay_id))
    f.write('\nEssay ids in testing:\n{}'.format(test_essay_id))

batches = zip(range(0, n_train-batch_size, batch_size), range(batch_size, n_train, batch_size))
batches = [(start, end) for start, end in batches]
print(batches)
x = 1
if x == 1:
    with tf.Graph().as_default():
        session_conf = tf.ConfigProto(
            allow_soft_placement=True,
            log_device_placement=False)

        global_step = tf.Variable(0, name="global_step", trainable=False)
        # decay learning rate
        starter_learning_rate = 0.0001
        learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 3000, 0.96, staircase=True)

        # test point
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, epsilon=0.1)
        #optimizer = tf.train.AdagradOptimizer(learning_rate)
        best_kappa_so_far = 0.0
        with tf.Session(config=session_conf) as sess:
            model = MemN2N_KV(batch_size, vocab_size, max_sent_size, max_sent_size, memory_size,
                              memory_size, embedding_size, len(score_range), feature_size, hops, reader, l2_lambda)

            grads_and_vars = optimizer.compute_gradients(model.loss_op, aggregation_method=tf.AggregationMethod.EXPERIMENTAL_TREE)
            grads_and_vars = [(tf.clip_by_norm(g, 10.0), v)
                              for g, v in grads_and_vars if g is not None]
            #grads_and_vars = [(add_gradient_noise(g, 1e-4), v) for g, v in grads_and_vars]
            train_op = optimizer.apply_gradients(grads_and_vars, name="train_op", global_step=global_step)
            sess.run(tf.global_variables_initializer(), feed_dict={model.w_placeholder: word2vec})
            saver = tf.train.Saver(tf.global_variables())

            for i in range(1, epochs+1):
                train_cost = 0
                total_time = 0
                np.random.shuffle(batches)
                for start, end in batches:
                    e = trainE[start:end]
                    s = train_scores_encoding[start:end]
                    s_num = train_scores[start:end]
                    #batched_memory = []
                    # batch sized memory
                    #for _ in range(len(e)):
                    #    batched_memory.append(memory)
                    mem_atten_encoding = []
                    for ite in s_num:
                        mem_encoding = np.zeros(memory_size)
                        for j_idx, j in enumerate(memory_score):
                            if j == ite:
                                mem_encoding[j_idx] = 1
                        mem_atten_encoding.append(mem_encoding)
                    batched_memory = [memory] * (end-start)
                    _, cost, time_spent = train_step(batched_memory, e, s, mem_atten_encoding)
                    total_time += time_spent
                    train_cost += cost
                print( 'Finish epoch {}, total training cost is {}, time spent is {}'.format(i, train_cost, total_time))
                # evaluation
                if i % 5 == 0 or i == 200:
                    # test on training data
                    train_preds = []
                    for start in range(0, n_train, test_batch_size):
                        end = min(n_train, start+test_batch_size)

                        #batched_memory = []
                        #for _ in range(end-start):
                        #    batched_memory.append(memory)
                        batched_memory = [memory] * (end-start)
#                         print("BM", len(batched_memory))
                        preds, _ = test_step(trainE[start:end], batched_memory)
                        if type(preds) is np.float32:
                            train_preds.append(preds)
                        else:
                            for ite in preds:
                                train_preds.append(ite)
                    if not is_regression:
                        train_preds = np.add(train_preds, min_score)
                    #train_kappp_score = kappa(train_scores, train_preds, 'quadratic')
                    train_kappp_score = quadratic_weighted_kappa(
                        train_scores, train_preds, min_score, max_score)
                    # test on test data
                    test_preds = []
                    test_atten_probs = []
                    for start in range(0, n_test, test_batch_size):
                        end = min(n_test, start+test_batch_size)

                        #batched_memory = []
                        #for _ in range(end-start):
                        #    batched_memory.append(memory)
                        batched_memory = [memory] * (end-start)
#                         print("Test", len(testE[start:end]))
                        
                        preds, mem_attention_probs = test_step(testE[start:end], batched_memory)
                        if type(preds) is np.float32:
                            test_preds.append(preds)
                        else:
                            for ite in preds:
                                test_preds.append(ite)
                        for ite in mem_attention_probs:
                            test_atten_probs.append(ite)
                    if not is_regression:
                        test_preds = np.add(test_preds, min_score)
                    #test_kappp_score = kappa(test_scores, test_preds, 'quadratic')
                    test_kappp_score = quadratic_weighted_kappa(
                        test_scores, test_preds, min_score, max_score)
                    stat_dict = {'pred_score': test_preds}
                    stat_df = pd.DataFrame(stat_dict)
                    # save the model if it gets best kappa
                    if(test_kappp_score > best_kappa_so_far):
                        early_stop_count = 0
                        best_kappa_so_far = test_kappp_score
                        # stats on test
                        stat_df.to_csv(out_dir+'/predScore_'+names)
                        with open(out_dir+'/mem_atten', 'a') as f:
                            for idx, ite in enumerate(test_essay_id):
                                f.write('{}\n'.format(ite))
                                f.write('{}\n'.format(test_atten_probs[idx]))
#                         saver.save(sess, out_dir+'/checkpoints', global_step)
#                         model.save('prompt1.h5')
#                         tf.saved_model.save(model, 'runs/')
                    else:
                        early_stop_count += 1
                    print("Training kappa score = {}".format(train_kappp_score))
                    print("Testing kappa score = {}".format(test_kappp_score))
                    with open(out_dir+'/eval_'.format(names), 'a') as f:
                        f.write("Training kappa score = {}\n".format(train_kappp_score))
                        f.write("Testing kappa score = {}\n".format(test_kappp_score))
                        f.write("Best Testing kappa score so far = {}\n".format(best_kappa_so_far))
                        f.write('*'*10)
                        f.write('\n')
                if early_stop_count > max_step_count:
                    break
            best_kappa_scores.append(best_kappa_so_far)

# model
with open(out_dir+'/eval_'.format(names), 'a') as f:
    f.write('5 fold cv {}\n'.format(best_kappa_scores))
    f.write('final result is {}'.format(np.mean(np.array(best_kappa_scores))))

#sys.stdout = orig_stdout
#f.close()


357
The size of training data: 1415
The size of testing data: 357
[(0, 15), (15, 30), (30, 45), (45, 60), (60, 75), (75, 90), (90, 105), (105, 120), (120, 135), (135, 150), (150, 165), (165, 180), (180, 195), (195, 210), (210, 225), (225, 240), (240, 255), (255, 270), (270, 285), (285, 300), (300, 315), (315, 330), (330, 345), (345, 360), (360, 375), (375, 390), (390, 405), (405, 420), (420, 435), (435, 450), (450, 465), (465, 480), (480, 495), (495, 510), (510, 525), (525, 540), (540, 555), (555, 570), (570, 585), (585, 600), (600, 615), (615, 630), (630, 645), (645, 660), (660, 675), (675, 690), (690, 705), (705, 720), (720, 735), (735, 750), (750, 765), (765, 780), (780, 795), (795, 810), (810, 825), (825, 840), (840, 855), (855, 870), (870, 885), (885, 900), (900, 915), (915, 930), (930, 945), (945, 960), (960, 975), (975, 990), (990, 1005), (1005, 1020), (1020, 1035), (1035, 1050), (1050, 1065), (1065, 1080), (1080, 1095), (1095, 1110), (1110, 1125), (1125, 1140), (1140, 1155), (1

KeyboardInterrupt: 

In [15]:
testE[start:end]

[[2734,
  458639,
  1,
  13,
  65,
  754,
  3352,
  17,
  23,
  1439,
  19,
  2,
  4510,
  6,
  2,
  94,
  15,
  1413,
  131,
  179,
  141,
  70,
  19,
  2597,
  79,
  94,
  271150,
  213,
  15,
  17,
  562,
  269,
  6300,
  4,
  7083,
  21,
  56,
  94,
  2756,
  2,
  1444,
  10,
  15,
  7,
  574,
  142,
  856,
  65,
  769,
  174,
  17,
  8,
  502,
  18,
  8184,
  17,
  24,
  2108,
  40,
  126,
  19,
  2,
  574,
  17,
  41,
  80,
  1476,
  1,
  1609,
  127,
  2537,
  19,
  2,
  885,
  4,
  1255,
  61,
  19,
  167,
  27,
  7,
  575,
  677195,
  10,
  646,
  27,
  42,
  13730,
  391,
  23,
  10,
  689,
  5,
  2,
  502,
  15,
  115,
  94,
  1266,
  4,
  116,
  1362,
  1767,
  115,
  6,
  106,
  13730,
  94,
  47,
  856,
  65,
  769,
  1767,
  133,
  45,
  25,
  16655,
  5,
  14903,
  574,
  176,
  15734,
  21,
  468,
  45,
  69,
  287,
  19,
  1902,
  1,
  115,
  1362,
  137,
  117,
  200,
  45,
  65,
  545,
  1902,
  50,
  36,
  156,
  5,
  13764,
  677195,
  27,
  170,
  42,
  13730,
  

In [3]:
import data_utils_adv
import numpy as np
from sklearn.model_selection import KFold
from qwk import quadratic_weighted_kappa
import tensorflow as tf
import time
import os
import sys
import pandas as pd

print( 'start to load flags\n')

# flags
# tf.flags.DEFINE_float("epsilon", 0.1, "Epsilon value for Adam Optimizer.")
# tf.flags.DEFINE_float("l2_lambda", 0.3, "Lambda for l2 loss.")
# tf.flags.DEFINE_float("learning_rate", 0.001, "Learning rate")
# tf.flags.DEFINE_float("max_grad_norm", 10.0, "Clip gradients to this norm.")
# tf.flags.DEFINE_float("keep_prob", 0.9, "Keep probability for dropout")
# tf.flags.DEFINE_integer("evaluation_interval", 5, "Evaluate and print results every x epochs")
# tf.flags.DEFINE_integer("batch_size", 15, "Batch size for training.")
# tf.flags.DEFINE_integer("feature_size", 100, "Feature size")
# tf.flags.DEFINE_integer("num_samples", 1, "Number of samples selected from training for each score")
# tf.flags.DEFINE_integer("hops", 3, "Number of hops in the Memory Network.")
# tf.flags.DEFINE_integer("epochs", 200, "Number of epochs to train for.")
# tf.flags.DEFINE_integer("embedding_size", 300, "Embedding size for embedding matrices.")
# tf.flags.DEFINE_integer("essay_set_id", 1, "essay set id, 1 <= id <= 8")
# tf.flags.DEFINE_integer("token_num", 42, "The number of token in glove (6, 42)")
# tf.flags.DEFINE_boolean("gated_addressing", False, "Simple gated addressing")
# tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
# tf.flags.DEFINE_boolean("is_regression", False, "The output is regression or classification")
# tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")
# # hyper-parameters
# FLAGS = tf.flags.FLAGS

early_stop_count = 0
max_step_count = 10
is_regression = False
gated_addressing = False
essay_set_id = 5
batch_size = 15
embedding_size = 300
feature_size = 100
l2_lambda = 0.3
hops = 3
reader = 'bow' # gru may not work
epochs = 200
num_samples = 1
num_tokens = 42
test_batch_size = batch_size
random_state = 0

if is_regression:
    from memn2n_kv_regression import MemN2N_KV
else:
    from memn2n_kv import MemN2N_KV
# print flags info
orig_stdout = sys.stdout
timestamp = time.strftime("%b_%d_%Y_%H:%M:%S", time.localtime())
folder_name = 'essay_set_{}_cv_{}_{}'.format(essay_set_id, num_samples, timestamp)
out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs/adversary_training/", folder_name))
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# save output to a file
#f = file(out_dir+'/out.txt', 'w')
#sys.stdout = f
print("Writing to {}\n".format(out_dir))

# print("\nParameters:")
# for key in sorted(FLAGS.__flags.keys()):
#     print("{}={}".format(key, getattr(FLAGS, key)))
# print("")

# with open(out_dir+'/params', 'w') as f:
#     for key in sorted(FLAGS.__flags.keys()):
#         f.write("{}={}".format(key, getattr(FLAGS, key)))
#         f.write("\n")

        
import os 
files = os.listdir('train_adv/prompt5/')
for testcase in files:
    training_path = 'train_adv/prompt5/'+testcase
    
    if testcase == 'noDisfluency&grammar_5_train_valid_reduce.csv':
        names = 'training_noDisGramm'
    elif testcase == 'noDisfluency_5_train_valid_reduce.csv':
        names = 'training_noDis'
    elif testcase == 'mixture_all_5_train_valid_reduce.csv':
        names = 'training_mixture'
    elif testcase == 'nocontractions_5_train_valid.csv':
        names = 'training_noContr'
    elif testcase == 'all_nochange_5_train_valid.csv':
        names = 'training_SynContr'
        
#     training_path = 'train_adv/prompt1/noDisfluency&grammar_1_train_valid_reduce.csv'
    essay_list, resolved_scores, essay_id = data_utils_adv.load_training_data(training_path, essay_set_id)

    # print(essay_id)
    testing_path = 'aes_data/essay5/fold_0/test.txt'
    essay_list_test, resolved_scores_test, essay_id_test = data_utils_adv.load_testing_data(testing_path, essay_set_id)

    max_score = max(resolved_scores)
    min_score = min(resolved_scores)
    if essay_set_id == 7:
        min_score, max_score = 0, 30
    elif essay_set_id == 8:
        min_score, max_score = 0, 60

    print( 'max_score is {} \t min_score is {}\n'.format(max_score, min_score))
    with open(out_dir+'/params', 'a') as f:
        f.write('max_score is {} \t min_score is {} \n'.format(max_score, min_score))
    
    score_range = range(min_score, max_score+1)

    #word_idx, _ = data_utils.build_vocab(essay_list, vocab_limit)

    # load glove
    word_idx, word2vec = data_utils_adv.load_glove(num_tokens, dim=embedding_size)

    vocab_size = len(word_idx) + 1
    # stat info on data set

    sent_size_list = list(map(len, [essay for essay in essay_list]))
    # print("sent size list", sent_size_list)
    max_sent_size = max(sent_size_list)
    mean_sent_size = int(np.mean(sent_size_list))

    print( 'max sentence size: {} \nmean sentence size: {}\n'.format(max_sent_size, mean_sent_size))
    with open(out_dir+'/params', 'a') as f:
        f.write('max sentence size: {} \nmean sentence size: {}\n'.format(max_sent_size, mean_sent_size))

    print( 'The length of score range is {}'.format(len(score_range)))
    E = data_utils_adv.vectorize_data(essay_list, word_idx, max_sent_size)

    labeled_data = zip(E, resolved_scores, sent_size_list)
    
    # For Testing
    sent_size_list_T = list(map(len, [essayT for essayT in essay_list_test]))
    max_sent_size_T = max(sent_size_list_T)
    mean_sent_size_T = int(np.mean(sent_size_list_T))

    # max_sent_size_T = max_sent_size_T.ljust(122, ' ')
    print( 'max sentence size: {} \nmean sentence size: {}\n'.format(max_sent_size_T, mean_sent_size_T))
    with open(out_dir+'/params', 'a') as f:
        f.write('max sentence size: {} \nmean sentence size: {}\n'.format(max_sent_size_T, mean_sent_size_T))

    # print( 'The length of score range of Testing Daat is {}'.format(len(score_range)))
    E_T = data_utils_adv.vectorize_data(essay_list_test, word_idx, max_sent_size_T)

    labeled_data_T = zip(E_T, resolved_scores_test, sent_size_list_T)
    
    def train_step(m, e, s, ma):
        start_time = time.time()
        feed_dict = {
            model._query: e,
            model._memory_key: m,
            model._score_encoding: s,
            model._mem_attention_encoding: ma,
            model.keep_prob: 0.9
            #model.w_placeholder: word2vec
        }
        _, step, predict_op, cost = sess.run([train_op, global_step, model.predict_op, model.cost], feed_dict)
        end_time = time.time()
        time_spent = end_time - start_time
        return predict_op, cost, time_spent

    def test_step(e, m):
        feed_dict = {
            model._query: e,
            model._memory_key: m,
            model.keep_prob: 1
            #model.w_placeholder: word2vec
        }
        preds, mem_attention_probs = sess.run([model.predict_op, model.mem_attention_probs], feed_dict)
        if is_regression:
            preds = np.clip(np.round(preds), min_score, max_score)
            return preds, mem_attention_probs
        else:
            return preds, mem_attention_probs
        fold_count = 0
    # kf = KFold(n_splits=2, random_state=random_state)
    early_stop_count = 0

    trainE = []
    train_scores = []
    train_essay_id = []
    best_kappa_scores = []

    testE = []
    test_scores = []
    test_essay_id = []

    print(len(E_T))
    # print(len(resolved_scores_test))
    # print(len(essay_list_test))
    for test_index in range(len(essay_id_test)):
        testE.append(E[test_index])
    #     print(len(testE))
        test_scores.append(resolved_scores_test[test_index])
        test_essay_id.append(essay_id_test[test_index])

    for train_index in range(len(essay_id)):
    #     print("Train_index", train_index)
    #     result = [int(x) for x, in train_index[0]]
    #     print(result)
    #     early_stop_count = 0
    #     fold_count += 1
    #     trainE = []
    #     testE = []
    #     train_scores = []
    #     test_scores = []
    #     train_essay_id = []
    #     test_essay_id = []

        trainE.append(E[train_index])
        train_scores.append(resolved_scores[train_index])
        train_essay_id.append(essay_id[train_index])

    #     for ite in train_index[0]:
    # #         print(ite)
    #         trainE.append(E[ite])
    #         train_scores.append(resolved_scores[ite])
    #         train_essay_id.append(essay_id[ite])
    #         print("Training Samples", E[ite], resolved_scores[ite], essay_id[ite])

    #     for ite in test_index:
    #         testE.append(E[ite])
    #         test_scores.append(resolved_scores[ite])
    #         test_essay_id.append(essay_id[ite])

    #trainE, testE, train_scores, test_scores, train_essay_id, test_essay_id = cross_validation.train_test_split(
    #    E, resolved_scores, essay_id, test_size=.2, random_state=random_state)

    memory = []
    memory_score = []
    memory_sent_size = []
    memory_essay_ids = []
    # pick sampled essay for each score
    for i in score_range:
    # test point: limit the number of samples in memory for 8
        for j in range(num_samples):
            if i in train_scores:
                score_idx = train_scores.index(i)
                score = train_scores.pop(score_idx)
                essay = trainE.pop(score_idx)
                sent_size = sent_size_list.pop(score_idx)
                memory.append(essay)
                memory_score.append(score)
                memory_essay_ids.append(train_essay_id.pop(score_idx))
                memory_sent_size.append(sent_size)
    memory_size = len(memory)
    if is_regression:
    # bad naming
        train_scores_encoding = train_scores
    else:
        train_scores_encoding = list(map(lambda x: score_range.index(x), train_scores))

        # data size
    n_train = len(trainE)
    n_test = len(testE)

    print( 'The size of training data: {}'.format(n_train))
    print( 'The size of testing data: {}'.format(n_test))
    with open(out_dir+'/params_{}'.format(names), 'a') as f:
        f.write('The size of training data: {}\n'.format(n_train))
        f.write('The size of testing data: {}\n'.format(n_test))
        f.write('\nEssay scores in memory:\n{}'.format(memory_score))
        f.write('\nEssay ids in memory:\n{}'.format(memory_essay_ids))
        f.write('\nEssay ids in training:\n{}'.format(train_essay_id))
        f.write('\nEssay ids in testing:\n{}'.format(test_essay_id))

    batches = zip(range(0, n_train-batch_size, batch_size), range(batch_size, n_train, batch_size))
    batches = [(start, end) for start, end in batches]
    print(batches)
    x = 1
    if x == 1:
        with tf.Graph().as_default():
            session_conf = tf.ConfigProto(
                allow_soft_placement=True,
                log_device_placement=False)

            global_step = tf.Variable(0, name="global_step", trainable=False)
            # decay learning rate
            starter_learning_rate = 0.0001
            learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 3000, 0.96, staircase=True)

            # test point
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, epsilon=0.1)
            #optimizer = tf.train.AdagradOptimizer(learning_rate)
            best_kappa_so_far = 0.0
            with tf.Session(config=session_conf) as sess:
                model = MemN2N_KV(batch_size, vocab_size, max_sent_size, max_sent_size, memory_size,
                                  memory_size, embedding_size, len(score_range), feature_size, hops, reader, l2_lambda)

                grads_and_vars = optimizer.compute_gradients(model.loss_op, aggregation_method=tf.AggregationMethod.EXPERIMENTAL_TREE)
                grads_and_vars = [(tf.clip_by_norm(g, 10.0), v)
                                  for g, v in grads_and_vars if g is not None]
                #grads_and_vars = [(add_gradient_noise(g, 1e-4), v) for g, v in grads_and_vars]
                train_op = optimizer.apply_gradients(grads_and_vars, name="train_op", global_step=global_step)
                sess.run(tf.global_variables_initializer(), feed_dict={model.w_placeholder: word2vec})
                saver = tf.train.Saver(tf.global_variables())

                for i in range(1, epochs+1):
                    train_cost = 0
                    total_time = 0
                    np.random.shuffle(batches)
                    for start, end in batches:
                        e = trainE[start:end]
                        s = train_scores_encoding[start:end]
                        s_num = train_scores[start:end]
                        #batched_memory = []
                        # batch sized memory
                        #for _ in range(len(e)):
                        #    batched_memory.append(memory)
                        mem_atten_encoding = []
                        for ite in s_num:
                            mem_encoding = np.zeros(memory_size)
                            for j_idx, j in enumerate(memory_score):
                                if j == ite:
                                    mem_encoding[j_idx] = 1
                            mem_atten_encoding.append(mem_encoding)
                        batched_memory = [memory] * (end-start)
                        _, cost, time_spent = train_step(batched_memory, e, s, mem_atten_encoding)
                        total_time += time_spent
                        train_cost += cost
                    print( 'Finish epoch {}, total training cost is {}, time spent is {}'.format(i, train_cost, total_time))
                    # evaluation
                    if i % 5 == 0 or i == 200:
                        # test on training data
                        train_preds = []
                        for start in range(0, n_train, test_batch_size):
                            end = min(n_train, start+test_batch_size)

                            #batched_memory = []
                            #for _ in range(end-start):
                            #    batched_memory.append(memory)
                            batched_memory = [memory] * (end-start)
    #                         print("BM", len(batched_memory))
                            preds, _ = test_step(trainE[start:end], batched_memory)
                            if type(preds) is np.float32:
                                train_preds.append(preds)
                            else:
                                for ite in preds:
                                    train_preds.append(ite)
                        if not is_regression:
                            train_preds = np.add(train_preds, min_score)
                        #train_kappp_score = kappa(train_scores, train_preds, 'quadratic')
                        train_kappp_score = quadratic_weighted_kappa(
                            train_scores, train_preds, min_score, max_score)
                        # test on test data
                        test_preds = []
                        test_atten_probs = []
                        for start in range(0, n_test, test_batch_size):
                            end = min(n_test, start+test_batch_size)

                            #batched_memory = []
                            #for _ in range(end-start):
                            #    batched_memory.append(memory)
                            batched_memory = [memory] * (end-start)
    #                         print("Test", len(testE[start:end]))

                            preds, mem_attention_probs = test_step(testE[start:end], batched_memory)
                            if type(preds) is np.float32:
                                test_preds.append(preds)
                            else:
                                for ite in preds:
                                    test_preds.append(ite)
                            for ite in mem_attention_probs:
                                test_atten_probs.append(ite)
                        if not is_regression:
                            test_preds = np.add(test_preds, min_score)
                        #test_kappp_score = kappa(test_scores, test_preds, 'quadratic')
                        test_kappp_score = quadratic_weighted_kappa(
                            test_scores, test_preds, min_score, max_score)
                        stat_dict = {'pred_score': test_preds}
                        stat_df = pd.DataFrame(stat_dict)
                        # save the model if it gets best kappa
                        if(test_kappp_score > best_kappa_so_far):
                            early_stop_count = 0
                            best_kappa_so_far = test_kappp_score
                            # stats on test
                            stat_df.to_csv(out_dir+'/predScore_'+names)
                            with open(out_dir+'/mem_atten', 'a') as f:
                                for idx, ite in enumerate(test_essay_id):
                                    f.write('{}\n'.format(ite))
                                    f.write('{}\n'.format(test_atten_probs[idx]))
    #                         saver.save(sess, out_dir+'/checkpoints', global_step)
    #                         model.save('prompt1.h5')
    #                         tf.saved_model.save(model, 'runs/')
                        else:
                            early_stop_count += 1
                        print("Training kappa score = {}".format(train_kappp_score))
                        print("Testing kappa score = {}".format(test_kappp_score))
                        with open(out_dir+'/eval_'.format(names), 'a') as f:
                            f.write("Training kappa score = {}\n".format(train_kappp_score))
                            f.write("Testing kappa score = {}\n".format(test_kappp_score))
                            f.write("Best Testing kappa score so far = {}\n".format(best_kappa_so_far))
                            f.write('*'*10)
                            f.write('\n')
                    if early_stop_count > max_step_count:
                        break
                best_kappa_scores.append(best_kappa_so_far)

    # model
    with open(out_dir+'/eval_'.format(names), 'a') as f:
        f.write('5 fold cv {}\n'.format(best_kappa_scores))
        f.write('final result is {}'.format(np.mean(np.array(best_kappa_scores))))

    #sys.stdout = orig_stdout
    #f.close()





start to load flags

Writing to /mnt/data/rajivratn/memory_networks/automated-essay-grading/runs/adversary_training/essay_set_5_cv_1_Mar_25_2020_15:36:12

                                               names  essay_id
0  2, "In this memoir of Narciso Rodriguez, @PERS...         1
1  1, The mood created by the author is showing h...         2
2  3, "The mood created by the author in the memo...         3
3  4, "In the memoir ""@PERSON1 Rodriguez"" from ...         4
4  2, "The author's interpretation of their child...         5
max_score is 4 	 min_score is 0

==> glove is loaded
max sentence size: 371 
mean sentence size: 114

The length of score range is 5
max sentence size: 452 
mean sentence size: 129

361
The size of training data: 1439
The size of testing data: 361
[(0, 15), (15, 30), (30, 45), (45, 60), (60, 75), (75, 90), (90, 105), (105, 120), (120, 135), (135, 150), (150, 165), (165, 180), (180, 195), (195, 210), (210, 225), (225, 240), (240, 255), (255, 270), (270, 285), (285

Finish epoch 59, total training cost is 2750.8058581352234, time spent is 1.1649034023284912
Finish epoch 60, total training cost is 3039.6970529556274, time spent is 1.1537415981292725
Training kappa score = 0.7989829984238559
Testing kappa score = 0.10997675207452573
Finish epoch 61, total training cost is 2968.6233139038086, time spent is 1.1748902797698975
Finish epoch 62, total training cost is 3008.0530219078064, time spent is 1.1389391422271729
Finish epoch 63, total training cost is 2689.3619718551636, time spent is 1.1304388046264648
Finish epoch 64, total training cost is 2669.113923072815, time spent is 1.1438019275665283
Finish epoch 65, total training cost is 2616.634388923645, time spent is 1.119722843170166
Training kappa score = 0.7983552764526223
Testing kappa score = 0.08824644333310483
Finish epoch 66, total training cost is 2411.1045427322388, time spent is 1.125469446182251
Finish epoch 67, total training cost is 2387.3488302230835, time spent is 1.1300559043884277

Finish epoch 43, total training cost is 3884.0906496047974, time spent is 1.2893686294555664
Finish epoch 44, total training cost is 4037.2582273483276, time spent is 1.2415132522583008
Finish epoch 45, total training cost is 4177.274675369263, time spent is 1.2682571411132812
Training kappa score = 0.7577709624045951
Testing kappa score = 0.07018775531695509
Finish epoch 46, total training cost is 3920.5119214057922, time spent is 1.270003080368042
Finish epoch 47, total training cost is 4022.1473412513733, time spent is 1.2505526542663574
Finish epoch 48, total training cost is 3949.310520172119, time spent is 1.2937395572662354
Finish epoch 49, total training cost is 3824.9885215759277, time spent is 1.323718786239624
Finish epoch 50, total training cost is 3890.7696323394775, time spent is 1.234682559967041
Training kappa score = 0.7665803018667833
Testing kappa score = -0.0013320164534305068
Finish epoch 51, total training cost is 3631.0906438827515, time spent is 1.28668546676635

Finish epoch 17, total training cost is 3460.8056983947754, time spent is 1.1590652465820312
Finish epoch 18, total training cost is 3649.5960187911987, time spent is 1.1729764938354492
Finish epoch 19, total training cost is 3106.1287207603455, time spent is 1.1293153762817383
Finish epoch 20, total training cost is 2769.5387427806854, time spent is 1.1634559631347656
Training kappa score = 0.8105224025735345
Testing kappa score = 0.00509279149943942
Finish epoch 21, total training cost is 2780.8136320114136, time spent is 1.1621601581573486
Finish epoch 22, total training cost is 2776.3474202156067, time spent is 1.1633827686309814
Finish epoch 23, total training cost is 2839.6686124801636, time spent is 1.1463675498962402
Finish epoch 24, total training cost is 2563.244955062866, time spent is 1.1388189792633057
Finish epoch 25, total training cost is 2565.3184604644775, time spent is 1.2020835876464844
Training kappa score = 0.753160362605503
Testing kappa score = -0.01308979326283

Finish epoch 11, total training cost is 7103.888006210327, time spent is 1.0278429985046387
Finish epoch 12, total training cost is 6744.615863800049, time spent is 1.0239388942718506
Finish epoch 13, total training cost is 6394.612607955933, time spent is 1.0308411121368408
Finish epoch 14, total training cost is 6337.051078796387, time spent is 1.025437593460083
Finish epoch 15, total training cost is 6056.854274749756, time spent is 1.026761770248413
Training kappa score = 0.6943610768816066
Testing kappa score = -0.03821729815283059
Finish epoch 16, total training cost is 5884.905466079712, time spent is 1.034196376800537
Finish epoch 17, total training cost is 5347.359408378601, time spent is 1.0250475406646729
Finish epoch 18, total training cost is 5815.326434135437, time spent is 1.0386936664581299
Finish epoch 19, total training cost is 5414.258594512939, time spent is 1.031217336654663
Finish epoch 20, total training cost is 5242.938688278198, time spent is 1.028792381286621


Finish epoch 1, total training cost is 173472.69305419922, time spent is 1.415177583694458
Finish epoch 2, total training cost is 37981.51354217529, time spent is 1.1124610900878906
Finish epoch 3, total training cost is 21728.71327972412, time spent is 1.2111601829528809
Finish epoch 4, total training cost is 17966.831520080566, time spent is 1.248978614807129
Finish epoch 5, total training cost is 15990.292465209961, time spent is 1.2252047061920166
Training kappa score = 0.6728224394698445
Testing kappa score = -0.001458694208202438
Finish epoch 6, total training cost is 15446.243965148926, time spent is 1.2411582469940186
Finish epoch 7, total training cost is 13065.088693618774, time spent is 1.24190354347229
Finish epoch 8, total training cost is 12806.755908966064, time spent is 1.235290288925171
Finish epoch 9, total training cost is 12158.926773071289, time spent is 1.2362499237060547
Finish epoch 10, total training cost is 11900.534423828125, time spent is 1.2983496189117432


Finish epoch 77, total training cost is 2364.9556846618652, time spent is 1.3646318912506104
Finish epoch 78, total training cost is 2222.6861991882324, time spent is 1.4344751834869385
Finish epoch 79, total training cost is 2134.178134918213, time spent is 1.405188798904419
Finish epoch 80, total training cost is 2247.467975616455, time spent is 1.3949832916259766
Training kappa score = 0.783613105444707
Testing kappa score = 0.05920372168650223
Finish epoch 81, total training cost is 2162.817478656769, time spent is 1.3345553874969482
Finish epoch 82, total training cost is 2125.5891585350037, time spent is 1.3557345867156982
Finish epoch 83, total training cost is 2179.651045322418, time spent is 1.3392808437347412
Finish epoch 84, total training cost is 1971.4283757209778, time spent is 1.3764870166778564
Finish epoch 85, total training cost is 2044.6514706611633, time spent is 1.3982179164886475
Training kappa score = 0.7919603521213532
Testing kappa score = 0.05295208294616216
F

Finish epoch 152, total training cost is 1295.6140217781067, time spent is 1.3938302993774414
Finish epoch 153, total training cost is 1324.0126214027405, time spent is 1.4780232906341553
Finish epoch 154, total training cost is 1316.3609342575073, time spent is 1.3768489360809326
Finish epoch 155, total training cost is 1282.31165933609, time spent is 1.459205150604248
Training kappa score = 0.8051946009314511
Testing kappa score = 0.03780290228733685
Finish epoch 156, total training cost is 1333.825602531433, time spent is 1.4404609203338623
Finish epoch 157, total training cost is 1317.5251786708832, time spent is 1.3605554103851318
Finish epoch 158, total training cost is 1303.8814315795898, time spent is 1.468928337097168
Finish epoch 159, total training cost is 1241.4993481636047, time spent is 1.3805360794067383
Finish epoch 160, total training cost is 1246.6632685661316, time spent is 1.4815809726715088
Training kappa score = 0.7992186409934421
Testing kappa score = 0.048199854

In [4]:
import data_utils_adv
import numpy as np
from sklearn.model_selection import KFold
from qwk import quadratic_weighted_kappa
import tensorflow as tf
import time
import os
import sys
import pandas as pd

print( 'start to load flags\n')

# flags
# tf.flags.DEFINE_float("epsilon", 0.1, "Epsilon value for Adam Optimizer.")
# tf.flags.DEFINE_float("l2_lambda", 0.3, "Lambda for l2 loss.")
# tf.flags.DEFINE_float("learning_rate", 0.001, "Learning rate")
# tf.flags.DEFINE_float("max_grad_norm", 10.0, "Clip gradients to this norm.")
# tf.flags.DEFINE_float("keep_prob", 0.9, "Keep probability for dropout")
# tf.flags.DEFINE_integer("evaluation_interval", 5, "Evaluate and print results every x epochs")
# tf.flags.DEFINE_integer("batch_size", 15, "Batch size for training.")
# tf.flags.DEFINE_integer("feature_size", 100, "Feature size")
# tf.flags.DEFINE_integer("num_samples", 1, "Number of samples selected from training for each score")
# tf.flags.DEFINE_integer("hops", 3, "Number of hops in the Memory Network.")
# tf.flags.DEFINE_integer("epochs", 200, "Number of epochs to train for.")
# tf.flags.DEFINE_integer("embedding_size", 300, "Embedding size for embedding matrices.")
# tf.flags.DEFINE_integer("essay_set_id", 1, "essay set id, 1 <= id <= 8")
# tf.flags.DEFINE_integer("token_num", 42, "The number of token in glove (6, 42)")
# tf.flags.DEFINE_boolean("gated_addressing", False, "Simple gated addressing")
# tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
# tf.flags.DEFINE_boolean("is_regression", False, "The output is regression or classification")
# tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")
# # hyper-parameters
# FLAGS = tf.flags.FLAGS

early_stop_count = 0
max_step_count = 10
is_regression = False
gated_addressing = False
essay_set_id = 6
batch_size = 15
embedding_size = 300
feature_size = 100
l2_lambda = 0.3
hops = 3
reader = 'bow' # gru may not work
epochs = 200
num_samples = 1
num_tokens = 42
test_batch_size = batch_size
random_state = 0

if is_regression:
    from memn2n_kv_regression import MemN2N_KV
else:
    from memn2n_kv import MemN2N_KV
# print flags info
orig_stdout = sys.stdout
timestamp = time.strftime("%b_%d_%Y_%H:%M:%S", time.localtime())
folder_name = 'essay_set_{}_cv_{}_{}'.format(essay_set_id, num_samples, timestamp)
out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs/adversary_training/", folder_name))
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# save output to a file
#f = file(out_dir+'/out.txt', 'w')
#sys.stdout = f
print("Writing to {}\n".format(out_dir))

# print("\nParameters:")
# for key in sorted(FLAGS.__flags.keys()):
#     print("{}={}".format(key, getattr(FLAGS, key)))
# print("")

# with open(out_dir+'/params', 'w') as f:
#     for key in sorted(FLAGS.__flags.keys()):
#         f.write("{}={}".format(key, getattr(FLAGS, key)))
#         f.write("\n")

        
import os 
files = os.listdir('train_adv/prompt6/')
for testcase in files:
    training_path = 'train_adv/prompt6/'+testcase
    
    if testcase == 'noDisfluency&grammar_6_train_valid_reduce.csv':
        names = 'training_noDisGramm'
    elif testcase == 'noDisfluency_6_train_valid_reduce.csv':
        names = 'training_noDis'
    elif testcase == 'mixture_all_6_train_valid_reduce.csv':
        names = 'training_mixture'
    elif testcase == 'nocontractions_6_train_valid.csv':
        names = 'training_noContr'
    elif testcase == 'all_nochange_6_train_valid.csv':
        names = 'training_SynContr'
        
#     training_path = 'train_adv/prompt1/noDisfluency&grammar_1_train_valid_reduce.csv'
    essay_list, resolved_scores, essay_id = data_utils_adv.load_training_data(training_path, essay_set_id)

    # print(essay_id)
    testing_path = 'aes_data/essay6/fold_0/test.txt'
    essay_list_test, resolved_scores_test, essay_id_test = data_utils_adv.load_testing_data(testing_path, essay_set_id)

    max_score = max(resolved_scores)
    min_score = min(resolved_scores)
    if essay_set_id == 7:
        min_score, max_score = 0, 30
    elif essay_set_id == 8:
        min_score, max_score = 0, 60

    print( 'max_score is {} \t min_score is {}\n'.format(max_score, min_score))
    with open(out_dir+'/params', 'a') as f:
        f.write('max_score is {} \t min_score is {} \n'.format(max_score, min_score))
    
    score_range = range(min_score, max_score+1)

    #word_idx, _ = data_utils.build_vocab(essay_list, vocab_limit)

    # load glove
    word_idx, word2vec = data_utils_adv.load_glove(num_tokens, dim=embedding_size)

    vocab_size = len(word_idx) + 1
    # stat info on data set

    sent_size_list = list(map(len, [essay for essay in essay_list]))
    # print("sent size list", sent_size_list)
    max_sent_size = max(sent_size_list)
    mean_sent_size = int(np.mean(sent_size_list))

    print( 'max sentence size: {} \nmean sentence size: {}\n'.format(max_sent_size, mean_sent_size))
    with open(out_dir+'/params', 'a') as f:
        f.write('max sentence size: {} \nmean sentence size: {}\n'.format(max_sent_size, mean_sent_size))

    print( 'The length of score range is {}'.format(len(score_range)))
    E = data_utils_adv.vectorize_data(essay_list, word_idx, max_sent_size)

    labeled_data = zip(E, resolved_scores, sent_size_list)
    
    # For Testing
    sent_size_list_T = list(map(len, [essayT for essayT in essay_list_test]))
    max_sent_size_T = max(sent_size_list_T)
    mean_sent_size_T = int(np.mean(sent_size_list_T))

    # max_sent_size_T = max_sent_size_T.ljust(122, ' ')
    print( 'max sentence size: {} \nmean sentence size: {}\n'.format(max_sent_size_T, mean_sent_size_T))
    with open(out_dir+'/params', 'a') as f:
        f.write('max sentence size: {} \nmean sentence size: {}\n'.format(max_sent_size_T, mean_sent_size_T))

    # print( 'The length of score range of Testing Daat is {}'.format(len(score_range)))
    E_T = data_utils_adv.vectorize_data(essay_list_test, word_idx, max_sent_size_T)

    labeled_data_T = zip(E_T, resolved_scores_test, sent_size_list_T)
    
    def train_step(m, e, s, ma):
        start_time = time.time()
        feed_dict = {
            model._query: e,
            model._memory_key: m,
            model._score_encoding: s,
            model._mem_attention_encoding: ma,
            model.keep_prob: 0.9
            #model.w_placeholder: word2vec
        }
        _, step, predict_op, cost = sess.run([train_op, global_step, model.predict_op, model.cost], feed_dict)
        end_time = time.time()
        time_spent = end_time - start_time
        return predict_op, cost, time_spent

    def test_step(e, m):
        feed_dict = {
            model._query: e,
            model._memory_key: m,
            model.keep_prob: 1
            #model.w_placeholder: word2vec
        }
        preds, mem_attention_probs = sess.run([model.predict_op, model.mem_attention_probs], feed_dict)
        if is_regression:
            preds = np.clip(np.round(preds), min_score, max_score)
            return preds, mem_attention_probs
        else:
            return preds, mem_attention_probs
        fold_count = 0
    # kf = KFold(n_splits=2, random_state=random_state)
    early_stop_count = 0

    trainE = []
    train_scores = []
    train_essay_id = []
    best_kappa_scores = []

    testE = []
    test_scores = []
    test_essay_id = []

    print(len(E_T))
    # print(len(resolved_scores_test))
    # print(len(essay_list_test))
    for test_index in range(len(essay_id_test)):
        testE.append(E[test_index])
    #     print(len(testE))
        test_scores.append(resolved_scores_test[test_index])
        test_essay_id.append(essay_id_test[test_index])

    for train_index in range(len(essay_id)):
    #     print("Train_index", train_index)
    #     result = [int(x) for x, in train_index[0]]
    #     print(result)
    #     early_stop_count = 0
    #     fold_count += 1
    #     trainE = []
    #     testE = []
    #     train_scores = []
    #     test_scores = []
    #     train_essay_id = []
    #     test_essay_id = []

        trainE.append(E[train_index])
        train_scores.append(resolved_scores[train_index])
        train_essay_id.append(essay_id[train_index])

    #     for ite in train_index[0]:
    # #         print(ite)
    #         trainE.append(E[ite])
    #         train_scores.append(resolved_scores[ite])
    #         train_essay_id.append(essay_id[ite])
    #         print("Training Samples", E[ite], resolved_scores[ite], essay_id[ite])

    #     for ite in test_index:
    #         testE.append(E[ite])
    #         test_scores.append(resolved_scores[ite])
    #         test_essay_id.append(essay_id[ite])

    #trainE, testE, train_scores, test_scores, train_essay_id, test_essay_id = cross_validation.train_test_split(
    #    E, resolved_scores, essay_id, test_size=.2, random_state=random_state)

    memory = []
    memory_score = []
    memory_sent_size = []
    memory_essay_ids = []
    # pick sampled essay for each score
    for i in score_range:
    # test point: limit the number of samples in memory for 8
        for j in range(num_samples):
            if i in train_scores:
                score_idx = train_scores.index(i)
                score = train_scores.pop(score_idx)
                essay = trainE.pop(score_idx)
                sent_size = sent_size_list.pop(score_idx)
                memory.append(essay)
                memory_score.append(score)
                memory_essay_ids.append(train_essay_id.pop(score_idx))
                memory_sent_size.append(sent_size)
    memory_size = len(memory)
    if is_regression:
    # bad naming
        train_scores_encoding = train_scores
    else:
        train_scores_encoding = list(map(lambda x: score_range.index(x), train_scores))

        # data size
    n_train = len(trainE)
    n_test = len(testE)

    print( 'The size of training data: {}'.format(n_train))
    print( 'The size of testing data: {}'.format(n_test))
    with open(out_dir+'/params_{}'.format(names), 'a') as f:
        f.write('The size of training data: {}\n'.format(n_train))
        f.write('The size of testing data: {}\n'.format(n_test))
        f.write('\nEssay scores in memory:\n{}'.format(memory_score))
        f.write('\nEssay ids in memory:\n{}'.format(memory_essay_ids))
        f.write('\nEssay ids in training:\n{}'.format(train_essay_id))
        f.write('\nEssay ids in testing:\n{}'.format(test_essay_id))

    batches = zip(range(0, n_train-batch_size, batch_size), range(batch_size, n_train, batch_size))
    batches = [(start, end) for start, end in batches]
    print(batches)
    x = 1
    if x == 1:
        with tf.Graph().as_default():
            session_conf = tf.ConfigProto(
                allow_soft_placement=True,
                log_device_placement=False)

            global_step = tf.Variable(0, name="global_step", trainable=False)
            # decay learning rate
            starter_learning_rate = 0.0001
            learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 3000, 0.96, staircase=True)

            # test point
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, epsilon=0.1)
            #optimizer = tf.train.AdagradOptimizer(learning_rate)
            best_kappa_so_far = 0.0
            with tf.Session(config=session_conf) as sess:
                model = MemN2N_KV(batch_size, vocab_size, max_sent_size, max_sent_size, memory_size,
                                  memory_size, embedding_size, len(score_range), feature_size, hops, reader, l2_lambda)

                grads_and_vars = optimizer.compute_gradients(model.loss_op, aggregation_method=tf.AggregationMethod.EXPERIMENTAL_TREE)
                grads_and_vars = [(tf.clip_by_norm(g, 10.0), v)
                                  for g, v in grads_and_vars if g is not None]
                #grads_and_vars = [(add_gradient_noise(g, 1e-4), v) for g, v in grads_and_vars]
                train_op = optimizer.apply_gradients(grads_and_vars, name="train_op", global_step=global_step)
                sess.run(tf.global_variables_initializer(), feed_dict={model.w_placeholder: word2vec})
                saver = tf.train.Saver(tf.global_variables())

                for i in range(1, epochs+1):
                    train_cost = 0
                    total_time = 0
                    np.random.shuffle(batches)
                    for start, end in batches:
                        e = trainE[start:end]
                        s = train_scores_encoding[start:end]
                        s_num = train_scores[start:end]
                        #batched_memory = []
                        # batch sized memory
                        #for _ in range(len(e)):
                        #    batched_memory.append(memory)
                        mem_atten_encoding = []
                        for ite in s_num:
                            mem_encoding = np.zeros(memory_size)
                            for j_idx, j in enumerate(memory_score):
                                if j == ite:
                                    mem_encoding[j_idx] = 1
                            mem_atten_encoding.append(mem_encoding)
                        batched_memory = [memory] * (end-start)
                        _, cost, time_spent = train_step(batched_memory, e, s, mem_atten_encoding)
                        total_time += time_spent
                        train_cost += cost
                    print( 'Finish epoch {}, total training cost is {}, time spent is {}'.format(i, train_cost, total_time))
                    # evaluation
                    if i % 5 == 0 or i == 200:
                        # test on training data
                        train_preds = []
                        for start in range(0, n_train, test_batch_size):
                            end = min(n_train, start+test_batch_size)

                            #batched_memory = []
                            #for _ in range(end-start):
                            #    batched_memory.append(memory)
                            batched_memory = [memory] * (end-start)
    #                         print("BM", len(batched_memory))
                            preds, _ = test_step(trainE[start:end], batched_memory)
                            if type(preds) is np.float32:
                                train_preds.append(preds)
                            else:
                                for ite in preds:
                                    train_preds.append(ite)
                        if not is_regression:
                            train_preds = np.add(train_preds, min_score)
                        #train_kappp_score = kappa(train_scores, train_preds, 'quadratic')
                        train_kappp_score = quadratic_weighted_kappa(
                            train_scores, train_preds, min_score, max_score)
                        # test on test data
                        test_preds = []
                        test_atten_probs = []
                        for start in range(0, n_test, test_batch_size):
                            end = min(n_test, start+test_batch_size)

                            #batched_memory = []
                            #for _ in range(end-start):
                            #    batched_memory.append(memory)
                            batched_memory = [memory] * (end-start)
    #                         print("Test", len(testE[start:end]))

                            preds, mem_attention_probs = test_step(testE[start:end], batched_memory)
                            if type(preds) is np.float32:
                                test_preds.append(preds)
                            else:
                                for ite in preds:
                                    test_preds.append(ite)
                            for ite in mem_attention_probs:
                                test_atten_probs.append(ite)
                        if not is_regression:
                            test_preds = np.add(test_preds, min_score)
                        #test_kappp_score = kappa(test_scores, test_preds, 'quadratic')
                        test_kappp_score = quadratic_weighted_kappa(
                            test_scores, test_preds, min_score, max_score)
                        stat_dict = {'pred_score': test_preds}
                        stat_df = pd.DataFrame(stat_dict)
                        # save the model if it gets best kappa
                        if(test_kappp_score > best_kappa_so_far):
                            early_stop_count = 0
                            best_kappa_so_far = test_kappp_score
                            # stats on test
                            stat_df.to_csv(out_dir+'/predScore_'+names)
                            with open(out_dir+'/mem_atten', 'a') as f:
                                for idx, ite in enumerate(test_essay_id):
                                    f.write('{}\n'.format(ite))
                                    f.write('{}\n'.format(test_atten_probs[idx]))
    #                         saver.save(sess, out_dir+'/checkpoints', global_step)
    #                         model.save('prompt1.h5')
    #                         tf.saved_model.save(model, 'runs/')
                        else:
                            early_stop_count += 1
                        print("Training kappa score = {}".format(train_kappp_score))
                        print("Testing kappa score = {}".format(test_kappp_score))
                        with open(out_dir+'/eval_'.format(names), 'a') as f:
                            f.write("Training kappa score = {}\n".format(train_kappp_score))
                            f.write("Testing kappa score = {}\n".format(test_kappp_score))
                            f.write("Best Testing kappa score so far = {}\n".format(best_kappa_so_far))
                            f.write('*'*10)
                            f.write('\n')
                    if early_stop_count > max_step_count:
                        break
                best_kappa_scores.append(best_kappa_so_far)

    # model
    with open(out_dir+'/eval_'.format(names), 'a') as f:
        f.write('5 fold cv {}\n'.format(best_kappa_scores))
        f.write('final result is {}'.format(np.mean(np.array(best_kappa_scores))))

    #sys.stdout = orig_stdout
    #f.close()





start to load flags

Writing to /mnt/data/rajivratn/memory_networks/automated-essay-grading/runs/adversary_training/essay_set_6_cv_1_Mar_25_2020_16:10:26

                                               names  essay_id
0  2, "There were many obstacles that the builder...         1
1  1, "In the passage The Mooring Mast by Marcia ...         2
2  3, The obstacles the builders face were great....         3
3  4, "While building the Empire State Building, ...         4
4  3, "Builders of the Empire State Building face...         5
max_score is 4 	 min_score is 0

==> glove is loaded
max sentence size: 497 
mean sentence size: 149

The length of score range is 5
max sentence size: 418 
mean sentence size: 156

360
The size of training data: 1435
The size of testing data: 360
[(0, 15), (15, 30), (30, 45), (45, 60), (60, 75), (75, 90), (90, 105), (105, 120), (120, 135), (135, 150), (150, 165), (165, 180), (180, 195), (195, 210), (210, 225), (225, 240), (240, 255), (255, 270), (270, 285), (285

Finish epoch 59, total training cost is 3131.666890144348, time spent is 1.5392062664031982
Finish epoch 60, total training cost is 2905.452236175537, time spent is 1.4069557189941406
Training kappa score = 0.6316201993057887
Testing kappa score = 0.05481171548117192
Finish epoch 61, total training cost is 2950.4787697792053, time spent is 1.5139482021331787
Finish epoch 62, total training cost is 3081.251805305481, time spent is 1.5364594459533691
Finish epoch 63, total training cost is 2859.178699016571, time spent is 1.531299114227295
Finish epoch 64, total training cost is 2730.209800004959, time spent is 1.5733888149261475
Finish epoch 65, total training cost is 3015.867064476013, time spent is 1.4680366516113281
Training kappa score = 0.6857727532626468
Testing kappa score = 0.0827023878858475
Finish epoch 66, total training cost is 3034.8584566116333, time spent is 1.470674991607666
Finish epoch 67, total training cost is 3080.430736541748, time spent is 1.4992783069610596
Finis

Finish epoch 38, total training cost is 4301.263426780701, time spent is 1.5182216167449951
Finish epoch 39, total training cost is 4108.064617156982, time spent is 1.4580650329589844
Finish epoch 40, total training cost is 3615.581630706787, time spent is 1.5044653415679932
Training kappa score = 0.6382632351208646
Testing kappa score = -0.07313410421985256
Finish epoch 41, total training cost is 3613.9499530792236, time spent is 1.490046501159668
Finish epoch 42, total training cost is 3954.2040281295776, time spent is 1.5390737056732178
Finish epoch 43, total training cost is 3887.3633422851562, time spent is 1.5434565544128418
Finish epoch 44, total training cost is 3661.968816757202, time spent is 1.5292844772338867
Finish epoch 45, total training cost is 3635.516655921936, time spent is 1.5078160762786865
Training kappa score = 0.6434786841752972
Testing kappa score = -0.08827270414652744
Finish epoch 46, total training cost is 3616.3947954177856, time spent is 1.5118181705474854

Finish epoch 37, total training cost is 4094.447693347931, time spent is 1.0542528629302979
Finish epoch 38, total training cost is 4070.273871898651, time spent is 1.0901556015014648
Finish epoch 39, total training cost is 4033.920313835144, time spent is 1.0656661987304688
Finish epoch 40, total training cost is 3839.652353286743, time spent is 1.084526538848877
Training kappa score = 0.7974814179199665
Testing kappa score = -0.01543374135178266
Finish epoch 41, total training cost is 3985.985689163208, time spent is 1.1954619884490967
Finish epoch 42, total training cost is 3779.2055315971375, time spent is 1.1862547397613525
Finish epoch 43, total training cost is 3716.2521181106567, time spent is 1.1834065914154053
Finish epoch 44, total training cost is 3514.840979576111, time spent is 1.1490166187286377
Finish epoch 45, total training cost is 3598.12975025177, time spent is 1.1355550289154053
Training kappa score = 0.778992761435392
Testing kappa score = 0.004358028830036775
Fin

Finish epoch 26, total training cost is 6635.303821563721, time spent is 1.5615010261535645
Finish epoch 27, total training cost is 6407.345380783081, time spent is 1.5936484336853027
Finish epoch 28, total training cost is 6396.937549591064, time spent is 1.566103458404541
Finish epoch 29, total training cost is 5981.424373626709, time spent is 1.5937881469726562
Finish epoch 30, total training cost is 6014.813238143921, time spent is 1.5524652004241943
Training kappa score = 0.6842953413419279
Testing kappa score = -0.07090232115978745
Finish epoch 31, total training cost is 5760.373719215393, time spent is 1.5472314357757568
Finish epoch 32, total training cost is 6262.245010375977, time spent is 1.5121705532073975
Finish epoch 33, total training cost is 5995.535252571106, time spent is 1.6373240947723389
Finish epoch 34, total training cost is 5987.09326171875, time spent is 1.6373391151428223
Finish epoch 35, total training cost is 5818.428031921387, time spent is 1.62035465240478

Training kappa score = 0.7204164367788519
Testing kappa score = -0.04546298014473904
Finish epoch 31, total training cost is 8497.028980255127, time spent is 1.344900131225586
Finish epoch 32, total training cost is 8266.655042648315, time spent is 1.3794763088226318
Finish epoch 33, total training cost is 8394.484655380249, time spent is 1.3464391231536865
Finish epoch 34, total training cost is 8766.467636108398, time spent is 1.371720314025879
Finish epoch 35, total training cost is 8265.612783908844, time spent is 1.3991289138793945
Training kappa score = 0.5970736779560308
Testing kappa score = -0.028215028215028415
Finish epoch 36, total training cost is 7978.009653091431, time spent is 1.3646271228790283
Finish epoch 37, total training cost is 7826.025053024292, time spent is 1.3626809120178223
Finish epoch 38, total training cost is 7170.438581466675, time spent is 1.3664445877075195
Finish epoch 39, total training cost is 7813.267788887024, time spent is 1.3618099689483643
Fin

In [5]:
import data_utils_adv
import numpy as np
from sklearn.model_selection import KFold
from qwk import quadratic_weighted_kappa
import tensorflow as tf
import time
import os
import sys
import pandas as pd

print( 'start to load flags\n')

# flags
# tf.flags.DEFINE_float("epsilon", 0.1, "Epsilon value for Adam Optimizer.")
# tf.flags.DEFINE_float("l2_lambda", 0.3, "Lambda for l2 loss.")
# tf.flags.DEFINE_float("learning_rate", 0.001, "Learning rate")
# tf.flags.DEFINE_float("max_grad_norm", 10.0, "Clip gradients to this norm.")
# tf.flags.DEFINE_float("keep_prob", 0.9, "Keep probability for dropout")
# tf.flags.DEFINE_integer("evaluation_interval", 5, "Evaluate and print results every x epochs")
# tf.flags.DEFINE_integer("batch_size", 15, "Batch size for training.")
# tf.flags.DEFINE_integer("feature_size", 100, "Feature size")
# tf.flags.DEFINE_integer("num_samples", 1, "Number of samples selected from training for each score")
# tf.flags.DEFINE_integer("hops", 3, "Number of hops in the Memory Network.")
# tf.flags.DEFINE_integer("epochs", 200, "Number of epochs to train for.")
# tf.flags.DEFINE_integer("embedding_size", 300, "Embedding size for embedding matrices.")
# tf.flags.DEFINE_integer("essay_set_id", 1, "essay set id, 1 <= id <= 8")
# tf.flags.DEFINE_integer("token_num", 42, "The number of token in glove (6, 42)")
# tf.flags.DEFINE_boolean("gated_addressing", False, "Simple gated addressing")
# tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
# tf.flags.DEFINE_boolean("is_regression", False, "The output is regression or classification")
# tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")
# # hyper-parameters
# FLAGS = tf.flags.FLAGS

early_stop_count = 0
max_step_count = 10
is_regression = False
gated_addressing = False
essay_set_id = 7
batch_size = 15
embedding_size = 300
feature_size = 100
l2_lambda = 0.3
hops = 3
reader = 'bow' # gru may not work
epochs = 200
num_samples = 1
num_tokens = 42
test_batch_size = batch_size
random_state = 0

if is_regression:
    from memn2n_kv_regression import MemN2N_KV
else:
    from memn2n_kv import MemN2N_KV
# print flags info
orig_stdout = sys.stdout
timestamp = time.strftime("%b_%d_%Y_%H:%M:%S", time.localtime())
folder_name = 'essay_set_{}_cv_{}_{}'.format(essay_set_id, num_samples, timestamp)
out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs/adversary_training/", folder_name))
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# save output to a file
#f = file(out_dir+'/out.txt', 'w')
#sys.stdout = f
print("Writing to {}\n".format(out_dir))

# print("\nParameters:")
# for key in sorted(FLAGS.__flags.keys()):
#     print("{}={}".format(key, getattr(FLAGS, key)))
# print("")

# with open(out_dir+'/params', 'w') as f:
#     for key in sorted(FLAGS.__flags.keys()):
#         f.write("{}={}".format(key, getattr(FLAGS, key)))
#         f.write("\n")

        
import os 
files = os.listdir('train_adv/prompt7/')
for testcase in files:
    training_path = 'train_adv/prompt7/'+testcase
    
    if testcase == 'noDisfluency&grammar_7_train_valid_reduce.csv':
        names = 'training_noDisGramm'
    elif testcase == 'noDisfluency_7_train_valid_reduce.csv':
        names = 'training_noDis'
    elif testcase == 'mixture_all_7_train_valid_reduce.csv':
        names = 'training_mixture'
    elif testcase == 'nocontractions_7_train_valid.csv':
        names = 'training_noContr'
    elif testcase == 'all_nochange_7_train_valid.csv':
        names = 'training_SynContr'
        
#     training_path = 'train_adv/prompt1/noDisfluency&grammar_1_train_valid_reduce.csv'
    essay_list, resolved_scores, essay_id = data_utils_adv.load_training_data(training_path, essay_set_id)

    # print(essay_id)
    testing_path = 'aes_data/essay7/fold_0/test.txt'
    essay_list_test, resolved_scores_test, essay_id_test = data_utils_adv.load_testing_data(testing_path, essay_set_id)

    max_score = max(resolved_scores)
    min_score = min(resolved_scores)
    if essay_set_id == 7:
        min_score, max_score = 0, 30
    elif essay_set_id == 8:
        min_score, max_score = 0, 60

    print( 'max_score is {} \t min_score is {}\n'.format(max_score, min_score))
    with open(out_dir+'/params', 'a') as f:
        f.write('max_score is {} \t min_score is {} \n'.format(max_score, min_score))
    
    score_range = range(min_score, max_score+1)

    #word_idx, _ = data_utils.build_vocab(essay_list, vocab_limit)

    # load glove
    word_idx, word2vec = data_utils_adv.load_glove(num_tokens, dim=embedding_size)

    vocab_size = len(word_idx) + 1
    # stat info on data set

    sent_size_list = list(map(len, [essay for essay in essay_list]))
    # print("sent size list", sent_size_list)
    max_sent_size = max(sent_size_list)
    mean_sent_size = int(np.mean(sent_size_list))

    print( 'max sentence size: {} \nmean sentence size: {}\n'.format(max_sent_size, mean_sent_size))
    with open(out_dir+'/params', 'a') as f:
        f.write('max sentence size: {} \nmean sentence size: {}\n'.format(max_sent_size, mean_sent_size))

    print( 'The length of score range is {}'.format(len(score_range)))
    E = data_utils_adv.vectorize_data(essay_list, word_idx, max_sent_size)

    labeled_data = zip(E, resolved_scores, sent_size_list)
    
    # For Testing
    sent_size_list_T = list(map(len, [essayT for essayT in essay_list_test]))
    max_sent_size_T = max(sent_size_list_T)
    mean_sent_size_T = int(np.mean(sent_size_list_T))

    # max_sent_size_T = max_sent_size_T.ljust(122, ' ')
    print( 'max sentence size: {} \nmean sentence size: {}\n'.format(max_sent_size_T, mean_sent_size_T))
    with open(out_dir+'/params', 'a') as f:
        f.write('max sentence size: {} \nmean sentence size: {}\n'.format(max_sent_size_T, mean_sent_size_T))

    # print( 'The length of score range of Testing Daat is {}'.format(len(score_range)))
    E_T = data_utils_adv.vectorize_data(essay_list_test, word_idx, max_sent_size_T)

    labeled_data_T = zip(E_T, resolved_scores_test, sent_size_list_T)
    
    def train_step(m, e, s, ma):
        start_time = time.time()
        feed_dict = {
            model._query: e,
            model._memory_key: m,
            model._score_encoding: s,
            model._mem_attention_encoding: ma,
            model.keep_prob: 0.9
            #model.w_placeholder: word2vec
        }
        _, step, predict_op, cost = sess.run([train_op, global_step, model.predict_op, model.cost], feed_dict)
        end_time = time.time()
        time_spent = end_time - start_time
        return predict_op, cost, time_spent

    def test_step(e, m):
        feed_dict = {
            model._query: e,
            model._memory_key: m,
            model.keep_prob: 1
            #model.w_placeholder: word2vec
        }
        preds, mem_attention_probs = sess.run([model.predict_op, model.mem_attention_probs], feed_dict)
        if is_regression:
            preds = np.clip(np.round(preds), min_score, max_score)
            return preds, mem_attention_probs
        else:
            return preds, mem_attention_probs
        fold_count = 0
    # kf = KFold(n_splits=2, random_state=random_state)
    early_stop_count = 0

    trainE = []
    train_scores = []
    train_essay_id = []
    best_kappa_scores = []

    testE = []
    test_scores = []
    test_essay_id = []

    print(len(E_T))
    # print(len(resolved_scores_test))
    # print(len(essay_list_test))
    for test_index in range(len(essay_id_test)):
        testE.append(E[test_index])
    #     print(len(testE))
        test_scores.append(resolved_scores_test[test_index])
        test_essay_id.append(essay_id_test[test_index])

    for train_index in range(len(essay_id)):
    #     print("Train_index", train_index)
    #     result = [int(x) for x, in train_index[0]]
    #     print(result)
    #     early_stop_count = 0
    #     fold_count += 1
    #     trainE = []
    #     testE = []
    #     train_scores = []
    #     test_scores = []
    #     train_essay_id = []
    #     test_essay_id = []

        trainE.append(E[train_index])
        train_scores.append(resolved_scores[train_index])
        train_essay_id.append(essay_id[train_index])

    #     for ite in train_index[0]:
    # #         print(ite)
    #         trainE.append(E[ite])
    #         train_scores.append(resolved_scores[ite])
    #         train_essay_id.append(essay_id[ite])
    #         print("Training Samples", E[ite], resolved_scores[ite], essay_id[ite])

    #     for ite in test_index:
    #         testE.append(E[ite])
    #         test_scores.append(resolved_scores[ite])
    #         test_essay_id.append(essay_id[ite])

    #trainE, testE, train_scores, test_scores, train_essay_id, test_essay_id = cross_validation.train_test_split(
    #    E, resolved_scores, essay_id, test_size=.2, random_state=random_state)

    memory = []
    memory_score = []
    memory_sent_size = []
    memory_essay_ids = []
    # pick sampled essay for each score
    for i in score_range:
    # test point: limit the number of samples in memory for 8
        for j in range(num_samples):
            if i in train_scores:
                score_idx = train_scores.index(i)
                score = train_scores.pop(score_idx)
                essay = trainE.pop(score_idx)
                sent_size = sent_size_list.pop(score_idx)
                memory.append(essay)
                memory_score.append(score)
                memory_essay_ids.append(train_essay_id.pop(score_idx))
                memory_sent_size.append(sent_size)
    memory_size = len(memory)
    if is_regression:
    # bad naming
        train_scores_encoding = train_scores
    else:
        train_scores_encoding = list(map(lambda x: score_range.index(x), train_scores))

        # data size
    n_train = len(trainE)
    n_test = len(testE)

    print( 'The size of training data: {}'.format(n_train))
    print( 'The size of testing data: {}'.format(n_test))
    with open(out_dir+'/params_{}'.format(names), 'a') as f:
        f.write('The size of training data: {}\n'.format(n_train))
        f.write('The size of testing data: {}\n'.format(n_test))
        f.write('\nEssay scores in memory:\n{}'.format(memory_score))
        f.write('\nEssay ids in memory:\n{}'.format(memory_essay_ids))
        f.write('\nEssay ids in training:\n{}'.format(train_essay_id))
        f.write('\nEssay ids in testing:\n{}'.format(test_essay_id))

    batches = zip(range(0, n_train-batch_size, batch_size), range(batch_size, n_train, batch_size))
    batches = [(start, end) for start, end in batches]
    print(batches)
    x = 1
    if x == 1:
        with tf.Graph().as_default():
            session_conf = tf.ConfigProto(
                allow_soft_placement=True,
                log_device_placement=False)

            global_step = tf.Variable(0, name="global_step", trainable=False)
            # decay learning rate
            starter_learning_rate = 0.0001
            learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 3000, 0.96, staircase=True)

            # test point
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, epsilon=0.1)
            #optimizer = tf.train.AdagradOptimizer(learning_rate)
            best_kappa_so_far = 0.0
            with tf.Session(config=session_conf) as sess:
                model = MemN2N_KV(batch_size, vocab_size, max_sent_size, max_sent_size, memory_size,
                                  memory_size, embedding_size, len(score_range), feature_size, hops, reader, l2_lambda)

                grads_and_vars = optimizer.compute_gradients(model.loss_op, aggregation_method=tf.AggregationMethod.EXPERIMENTAL_TREE)
                grads_and_vars = [(tf.clip_by_norm(g, 10.0), v)
                                  for g, v in grads_and_vars if g is not None]
                #grads_and_vars = [(add_gradient_noise(g, 1e-4), v) for g, v in grads_and_vars]
                train_op = optimizer.apply_gradients(grads_and_vars, name="train_op", global_step=global_step)
                sess.run(tf.global_variables_initializer(), feed_dict={model.w_placeholder: word2vec})
                saver = tf.train.Saver(tf.global_variables())

                for i in range(1, epochs+1):
                    train_cost = 0
                    total_time = 0
                    np.random.shuffle(batches)
                    for start, end in batches:
                        e = trainE[start:end]
                        s = train_scores_encoding[start:end]
                        s_num = train_scores[start:end]
                        #batched_memory = []
                        # batch sized memory
                        #for _ in range(len(e)):
                        #    batched_memory.append(memory)
                        mem_atten_encoding = []
                        for ite in s_num:
                            mem_encoding = np.zeros(memory_size)
                            for j_idx, j in enumerate(memory_score):
                                if j == ite:
                                    mem_encoding[j_idx] = 1
                            mem_atten_encoding.append(mem_encoding)
                        batched_memory = [memory] * (end-start)
                        _, cost, time_spent = train_step(batched_memory, e, s, mem_atten_encoding)
                        total_time += time_spent
                        train_cost += cost
                    print( 'Finish epoch {}, total training cost is {}, time spent is {}'.format(i, train_cost, total_time))
                    # evaluation
                    if i % 5 == 0 or i == 200:
                        # test on training data
                        train_preds = []
                        for start in range(0, n_train, test_batch_size):
                            end = min(n_train, start+test_batch_size)

                            #batched_memory = []
                            #for _ in range(end-start):
                            #    batched_memory.append(memory)
                            batched_memory = [memory] * (end-start)
    #                         print("BM", len(batched_memory))
                            preds, _ = test_step(trainE[start:end], batched_memory)
                            if type(preds) is np.float32:
                                train_preds.append(preds)
                            else:
                                for ite in preds:
                                    train_preds.append(ite)
                        if not is_regression:
                            train_preds = np.add(train_preds, min_score)
                        #train_kappp_score = kappa(train_scores, train_preds, 'quadratic')
                        train_kappp_score = quadratic_weighted_kappa(
                            train_scores, train_preds, min_score, max_score)
                        # test on test data
                        test_preds = []
                        test_atten_probs = []
                        for start in range(0, n_test, test_batch_size):
                            end = min(n_test, start+test_batch_size)

                            #batched_memory = []
                            #for _ in range(end-start):
                            #    batched_memory.append(memory)
                            batched_memory = [memory] * (end-start)
    #                         print("Test", len(testE[start:end]))

                            preds, mem_attention_probs = test_step(testE[start:end], batched_memory)
                            if type(preds) is np.float32:
                                test_preds.append(preds)
                            else:
                                for ite in preds:
                                    test_preds.append(ite)
                            for ite in mem_attention_probs:
                                test_atten_probs.append(ite)
                        if not is_regression:
                            test_preds = np.add(test_preds, min_score)
                        #test_kappp_score = kappa(test_scores, test_preds, 'quadratic')
                        test_kappp_score = quadratic_weighted_kappa(
                            test_scores, test_preds, min_score, max_score)
                        stat_dict = {'pred_score': test_preds}
                        stat_df = pd.DataFrame(stat_dict)
                        # save the model if it gets best kappa
                        if(test_kappp_score > best_kappa_so_far):
                            early_stop_count = 0
                            best_kappa_so_far = test_kappp_score
                            # stats on test
                            stat_df.to_csv(out_dir+'/predScore_'+names)
                            with open(out_dir+'/mem_atten', 'a') as f:
                                for idx, ite in enumerate(test_essay_id):
                                    f.write('{}\n'.format(ite))
                                    f.write('{}\n'.format(test_atten_probs[idx]))
    #                         saver.save(sess, out_dir+'/checkpoints', global_step)
    #                         model.save('prompt1.h5')
    #                         tf.saved_model.save(model, 'runs/')
                        else:
                            early_stop_count += 1
                        print("Training kappa score = {}".format(train_kappp_score))
                        print("Testing kappa score = {}".format(test_kappp_score))
                        with open(out_dir+'/eval_'.format(names), 'a') as f:
                            f.write("Training kappa score = {}\n".format(train_kappp_score))
                            f.write("Testing kappa score = {}\n".format(test_kappp_score))
                            f.write("Best Testing kappa score so far = {}\n".format(best_kappa_so_far))
                            f.write('*'*10)
                            f.write('\n')
                    if early_stop_count > max_step_count:
                        break
                best_kappa_scores.append(best_kappa_so_far)

    # model
    with open(out_dir+'/eval_'.format(names), 'a') as f:
        f.write('5 fold cv {}\n'.format(best_kappa_scores))
        f.write('final result is {}'.format(np.mean(np.array(best_kappa_scores))))

    #sys.stdout = orig_stdout
    #f.close()





start to load flags

Writing to /mnt/data/rajivratn/memory_networks/automated-essay-grading/runs/adversary_training/essay_set_7_cv_1_Mar_25_2020_16:46:26

                                               names  essay_id
0  17, "I going to write about a time when I went...         1
1  12, "One time I was patience it was when I wan...         2
2  14, "The time that I was patient was not long ...         3
3  17, "Im writing about the time I was patient a...         4
4  15, "Patience is when you are waiting for some...         5
max_score is 30 	 min_score is 0

==> glove is loaded
max sentence size: 671 
mean sentence size: 156

The length of score range is 31
max sentence size: 474 
mean sentence size: 176

314
The size of training data: 1232
The size of testing data: 314
[(0, 15), (15, 30), (30, 45), (45, 60), (60, 75), (75, 90), (90, 105), (105, 120), (120, 135), (135, 150), (150, 165), (165, 180), (180, 195), (195, 210), (210, 225), (225, 240), (240, 255), (255, 270), (270, 285), (2

Training kappa score = 0.4027103547614065
Testing kappa score = 0.07355573804023108
Finish epoch 61, total training cost is 3724.9933853149414, time spent is 4.126752614974976
Finish epoch 62, total training cost is 3803.0560913085938, time spent is 4.102953195571899
Finish epoch 63, total training cost is 3818.1110038757324, time spent is 4.114755630493164
Finish epoch 64, total training cost is 3791.4207038879395, time spent is 4.221102476119995
Finish epoch 65, total training cost is 3864.5725288391113, time spent is 4.18124794960022
Training kappa score = 0.46455322502843
Testing kappa score = 0.018711082242807153
Finish epoch 66, total training cost is 3872.540382385254, time spent is 4.1348466873168945
Finish epoch 67, total training cost is 3922.044403076172, time spent is 4.2014853954315186
Finish epoch 68, total training cost is 3744.005672454834, time spent is 4.229610443115234
Finish epoch 69, total training cost is 3657.995620727539, time spent is 4.354031801223755
Finish e

Finish epoch 6, total training cost is 34785.67184448242, time spent is 3.9219610691070557
Finish epoch 7, total training cost is 31546.91323852539, time spent is 3.6396896839141846
Finish epoch 8, total training cost is 29043.278915405273, time spent is 3.9607186317443848
Finish epoch 9, total training cost is 27869.665451049805, time spent is 3.9315459728240967
Finish epoch 10, total training cost is 25432.110870361328, time spent is 3.6106555461883545
Training kappa score = 0.30723072022662046
Testing kappa score = 0.02077266219800955
Finish epoch 11, total training cost is 24102.97640991211, time spent is 3.7751681804656982
Finish epoch 12, total training cost is 22112.002471923828, time spent is 3.851398468017578
Finish epoch 13, total training cost is 22005.22523498535, time spent is 3.5266153812408447
Finish epoch 14, total training cost is 21169.799758911133, time spent is 3.85526180267334
Finish epoch 15, total training cost is 20026.70458984375, time spent is 3.56371927261352

Finish epoch 82, total training cost is 7750.639678955078, time spent is 4.407215356826782
Finish epoch 83, total training cost is 7731.642803192139, time spent is 4.493838548660278
Finish epoch 84, total training cost is 7545.045570373535, time spent is 4.450666427612305
Finish epoch 85, total training cost is 7480.469806671143, time spent is 4.50229024887085
Training kappa score = 0.6186165975650542
Testing kappa score = 0.011464736237582929
Finish epoch 86, total training cost is 7252.374355316162, time spent is 4.417397975921631
Finish epoch 87, total training cost is 7393.136589050293, time spent is 4.451233386993408
Finish epoch 88, total training cost is 7316.365749359131, time spent is 4.466912269592285
Finish epoch 89, total training cost is 7507.611297607422, time spent is 4.341086387634277
Finish epoch 90, total training cost is 7084.871315002441, time spent is 4.359257698059082
Training kappa score = 0.5963810002513196
Testing kappa score = 0.03570471321077995
Finish epoch 

Finish epoch 43, total training cost is 3145.0393676757812, time spent is 2.9377644062042236
Finish epoch 44, total training cost is 3116.6124267578125, time spent is 2.9420840740203857
Finish epoch 45, total training cost is 3079.987232208252, time spent is 2.9367010593414307
Training kappa score = 0.667396651077204
Testing kappa score = 0.08792848927775943
Finish epoch 46, total training cost is 3073.6317176818848, time spent is 3.0624001026153564
Finish epoch 47, total training cost is 3086.739070892334, time spent is 2.973160982131958
Finish epoch 48, total training cost is 3068.588535308838, time spent is 2.9452667236328125
Finish epoch 49, total training cost is 3001.579484939575, time spent is 3.0578155517578125
Finish epoch 50, total training cost is 2958.3868980407715, time spent is 2.9917752742767334
Training kappa score = 0.6618401021161694
Testing kappa score = 0.08893508552834528
Finish epoch 51, total training cost is 2966.9284172058105, time spent is 2.8602206707000732
F

Finish epoch 14, total training cost is 25973.388214111328, time spent is 3.8016529083251953
Finish epoch 15, total training cost is 24485.328582763672, time spent is 3.7289929389953613
Training kappa score = 0.24779449834674216
Testing kappa score = 0.0020665367533473367
Finish epoch 16, total training cost is 23682.577713012695, time spent is 4.055936098098755
Finish epoch 17, total training cost is 23147.363555908203, time spent is 3.865281105041504
Finish epoch 18, total training cost is 22391.457397460938, time spent is 3.7681193351745605
Finish epoch 19, total training cost is 22079.16438293457, time spent is 3.80971097946167
Finish epoch 20, total training cost is 21091.307250976562, time spent is 3.850612163543701
Training kappa score = 0.3070223477058006
Testing kappa score = -0.05141492444759832
Finish epoch 21, total training cost is 20698.900848388672, time spent is 3.7395622730255127
Finish epoch 22, total training cost is 20280.40461730957, time spent is 3.784509420394897

Finish epoch 1, total training cost is 171016.28448486328, time spent is 3.706602096557617
Finish epoch 2, total training cost is 81651.0484008789, time spent is 3.81820011138916
Finish epoch 3, total training cost is 61514.73165893555, time spent is 3.82863712310791
Finish epoch 4, total training cost is 54160.086669921875, time spent is 3.4851162433624268
Finish epoch 5, total training cost is 49437.29476928711, time spent is 3.7843129634857178
Training kappa score = 0.383461468752862
Testing kappa score = -0.016640893684599645
Finish epoch 6, total training cost is 42749.50100708008, time spent is 2.920825719833374
Finish epoch 7, total training cost is 38023.21376037598, time spent is 2.944030284881592
Finish epoch 8, total training cost is 34963.724197387695, time spent is 2.9228172302246094
Finish epoch 9, total training cost is 33171.61065673828, time spent is 2.9184091091156006
Finish epoch 10, total training cost is 31286.024963378906, time spent is 2.9079694747924805
Training

Finish epoch 77, total training cost is 5928.319274902344, time spent is 3.8080549240112305
Finish epoch 78, total training cost is 5850.366592407227, time spent is 3.8966474533081055
Finish epoch 79, total training cost is 5295.135257720947, time spent is 3.9524412155151367
Finish epoch 80, total training cost is 5025.569606781006, time spent is 4.035022497177124
Training kappa score = 0.2351620045316889
Testing kappa score = -0.018046523123638858
